In [13]:
# # Dependencies
# import numpy as np
# import pandas as pd
# import matplotlib.pyplot as plt
# import requests
# from census import Census

# # Census API Key
# from config import api_key
# c = Census(api_key, year=2013)

import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
from scipy.stats import linregress
import scipy.stats as st
from datetime import date
from functools import reduce
import json
import ctypes
import seaborn as sns
import datetime

In [14]:
# # Run Census Search to retrieve data on all zip codes (2013 ACS5 Census)
# # See: https://github.com/CommerceDataService/census-wrapper for library documentation
# # See: https://gist.github.com/afhaque/60558290d6efd892351c4b64e5c01e9b for labels
# census_data = c.acs5.get(("NAME", "B19013_001E", "B01003_001E", "B01002_001E",
#                           "B19301_001E",
#                           "B17001_002E"), {'for': 'zip code tabulation area:*'})

# # Convert to DataFrame
# census_pd = pd.DataFrame(census_data)

# # Column Reordering
# census_pd = census_pd.rename(columns={"B01003_001E": "Population",
#                                       "B01002_001E": "Median Age",
#                                       "B19013_001E": "Household Income",
#                                       "B19301_001E": "Per Capita Income",
#                                       "B17001_002E": "Poverty Count",
#                                       "NAME": "Name", "zip code tabulation area": "Zipcode"})

# # Add in Poverty Rate (Poverty Count / Population)
# census_pd["Poverty Rate"] = 100 * \
#     census_pd["Poverty Count"].astype(
#         int) / census_pd["Population"].astype(int)

# # Final DataFrame
# census_pd = census_pd[["Zipcode", "Population", "Median Age", "Household Income",
#                        "Per Capita Income", "Poverty Count", "Poverty Rate"]]

# # Visualize
# print(len(census_pd))
# census_pd.head()

In [15]:
response = requests.get("https://api.census.gov/data/2018/acs/acs1?get=NAME,B06011_001E&for=county:*").json()
                        
# locate the first row and make it the column name and then drop row
income_df = pd.DataFrame(response)
income_df.columns = income_df.iloc[0]
income_df = income_df.drop([1])
    
# delete and null info and reset the index
income_df = income_df.dropna(how='any')
income_df.reset_index()
    
#replace the integer column names with values in row 0
income_df.columns = income_df.iloc[0]
income_df = income_df.drop([0])
income_df.reset_index()
income_df

,NAME,B06011_001E,state,county
2,"Calhoun County, Alabama",22154,01,015
3,"Cullman County, Alabama",23615,01,043
4,"DeKalb County, Alabama",22130,01,049
5,"Elmore County, Alabama",27093,01,051
6,"Etowah County, Alabama",25055,01,055
...,...,...,...,...
822,"Waukesha County, Wisconsin",40275,55,133
823,"Winnebago County, Wisconsin",30714,55,139
824,"Wood County, Wisconsin",31739,55,141
825,"Laramie County, Wyoming",32603,56,021


In [16]:
# create new columns for County and State
# ----Remove Columns: NAME, state, county
#-----Change column names to match merged UCR Crime data
#-----Remove word "County" in County name
income_df['County Name'] = income_df['NAME'].str.split(', ').apply(lambda x: x[0])
income_df['State Name'] = income_df['NAME'].str.split(', ').apply(lambda x: x[-1])
income_df['County Name'] = income_df['NAME'].str.split(' ').apply(lambda x: x[0])
income_df.drop(columns = ["NAME", "state", "county"], inplace = True)
income_df.rename(columns={"County Name":"COUNTY", "State Name":"STATE", "B06011_001E":"MEDIAN INCOME"}, inplace = True)
                          
# Clean up formating in Median Income
income_df["MEDIAN INCOME"] = income_df["MEDIAN INCOME"].apply("int64")
income_df

,MEDIAN INCOME,COUNTY,STATE
2,22154,Calhoun,Alabama
3,23615,Cullman,Alabama
4,22130,DeKalb,Alabama
5,27093,Elmore,Alabama
6,25055,Etowah,Alabama
...,...,...,...
822,40275,Waukesha,Wisconsin
823,30714,Winnebago,Wisconsin
824,31739,Wood,Wisconsin
825,32603,Laramie,Wyoming


In [17]:
response = requests.get(f"https://api.census.gov/data/2017/pep/population?get=DENSITY,POP,STATE&for=county:*").json()

# locate the first row and make it the column name and then drop row
county_dens_df = pd.DataFrame(response)
county_dens_df.columns = county_dens_df.iloc[0]
county_dens_df = county_dens_df.drop([0])
    
county_dens_df["state"] = county_dens_df["state"].apply(str)
county_dens_df["county"] = county_dens_df["county"].apply(str)
county_dens_df["county"] = county_dens_df["county"].apply(lambda x: x.zfill(3))
county_dens_df["CODE"] = county_dens_df[["state", "county"]].agg("".join, axis=1)

# delete and null info and reset the index
county_dens_df = county_dens_df.dropna(how='any')
county_dens_df

,DENSITY,POP,STATE,state,county,CODE
1,14.666355113,12469,01,01,035,01035
2,16.521468381,10754,01,01,037,01037
3,35.995681469,37092,01,01,039,01039
4,22.781736369,13871,01,01,041,01041
5,112.62558771,82755,01,01,043,01043
...,...,...,...,...,...,...
3210,29.883159282,17516,55,55,103,55103
3211,226.01232055,162309,55,55,105,55105
3212,15.490065234,14151,55,55,107,55107
3213,122.77496179,88703,55,55,109,55109


In [2]:
# Save as a csv
# Note to avoid any issues later, use encoding="utf-8"



# census_pd.to_csv("census_data.csv", encoding="utf-8", index=False)

NameError: name 'census_pd' is not defined